In [1]:
from collections import deque

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

np.set_printoptions(threshold=np.inf)

In [31]:
directions = {(-1, 0): 'N', (1, 0): 'S', (0, -1): 'W', (0, 1): 'E'}
directions_reversed = {'N': (1, 0), 'S': (-1, 0), 'W': (0, 1), 'E': (0, -1)}
opposite_directions = {('N', 'S'), ('S', 'N'), ('E', 'W'), ('W', 'E')}

In [3]:
def parse_input(file):
    with open(file) as file_in:
        grid = file_in.read().splitlines()

    grid = np.array([list(row) for row in grid])

    S_coords = tuple(np.argwhere(grid == 'S')[0].tolist())
    E_coords = tuple(np.argwhere(grid == 'E')[0].tolist())
    dot_coords = set([(x.item(), y.item()) for x, y in np.argwhere(grid == '.')])
    wall_coords = set([(x.item(), y.item()) for x, y in np.argwhere(grid == '#')])

    return S_coords, E_coords, wall_coords

In [4]:
def get_best_states(S_coords, E_coords, wall_coords):
    start = (S_coords, 'E', 0)
    queue = deque([start])
    scores = []
    best_states = {}
    min_score = float('inf')

    while queue:
        (x_current, y_current), dir_current, score = queue.popleft()

        # Stop exploring paths with scores exceeding the minimum score
        if score > min_score:
            continue

        # End condition
        if (x_current, y_current) == E_coords:
            min_score = min(min_score, score + 1)
            continue

        for (dx, dy), dir_next in directions.items():
            x_next, y_next = x_current + dx, y_current + dy 

            if (x_next, y_next) in wall_coords:
                continue

            new_score = score + 1 if dir_next == dir_current else score + 1001

            state = ((x_next, y_next), dir_next)
            if state not in best_states or new_score < best_states[state]:
                best_states[state] = new_score
                queue.append(((x_next, y_next), dir_next, new_score))

    return best_states

In [5]:
def main1(file):
    S_coords, E_coords, wall_coords = parse_input(file)
    best_states = get_best_states(S_coords, E_coords, wall_coords)
    end_scores= [best_states[((x, y), dir)] for (x, y), dir in best_states if (x, y) == E_coords]
    return min(end_scores)

In [6]:
assert main1('example1.txt') == 7036
assert main1('example2.txt') == 11048

In [7]:
main1('input.txt')

102460

In [37]:
def main2(file):
    S_coords, E_coords, wall_coords = parse_input(file)
    best_states = get_best_states(S_coords, E_coords, wall_coords)

    # Get end state with minimal score
    end_scores = {((x, y), dir): score for ((x, y), dir), score in best_states.items() if (x, y) == E_coords}
    end_state_min_score = min(end_scores, key=end_scores.get)

    # Reversed BFS from end to start to get all nodes that are on an optimal path
    tiles_best_paths = set()
    visited = set()
    queue = deque([end_state_min_score])
    while queue:
        (x_current, y_current), dir_current = queue.popleft()
        tiles_best_paths.add((x_current, y_current))
        dx, dy = directions_reversed[dir_current]
        x_next, y_next = x_current + dx, y_current + dy
        candidates = []
        for (x, y), dir in best_states:
            if (x, y) == (x_next, y_next) and ((x, y), dir) not in visited:
                diff_score = best_states[((x_current, y_current), dir_current)] - best_states[((x, y), dir)]
                if diff_score == 1 or diff_score == 1001:
                    candidates.append(((x, y), dir))
        queue.extend(candidates)
        visited.update(candidates)
    tiles_best_paths.add((x_next, y_next))

    return len(tiles_best_paths)

In [41]:
main2('input.txt') 

527